# Extracción de categorías del libro.

In [ ]:
import requests
from bs4 import BeautifulSoup

# Defino la base del sitio principal
BASE_URL = "https://books.toscrape.com/"

# Creo la URL inicial del sitio y uso BASE_URL + url_relativa para armar URLs completas de las categorías
CATEGORIAS_URL = BASE_URL + "index.html" 

# Extraigo con el método get las categorias y con el "parser" obtengo la info relevante
respuesta = requests.get(CATEGORIAS_URL)
soup = BeautifulSoup(respuesta.text, "html.parser")

# Extaigo los enlaces de categorias (hay uno para cada categoria dentro de 'ul.nav-list')
categorias_links = soup.select("ul.nav-list ul li a")

categoria = {} # Creo un diccionario vacio para guardar categorias

for link in categorias_links: 
    nombre = link.text.strip() # Obtengo y limpio el nombre de la categoria
    url_relativa = link['href'] # Saco la URL relativa del href
    url_completa = BASE_URL + url_relativa # Armo la URL completa
    categoria[nombre] = url_completa # Guardo en el diccionario: nombre -> URL

print(f"Categorias encontradas: {len(categoria)}\n")

# Recorre cada par del diccionario : (clave,valor) y con el .items devuelve la lista de pares
for nombre, url in categoria.items():
    print(f"- {nombre}: {url}")
    




# Obtención de libros por categoría


In [4]:
import time # Para no hacer pedidos(requests) una tras otra, porque puede causar problemas

"""Creo una función y le doy dos argumentos"""
def obtener_libros_por_categoria(nombre_categoria,url_categoria):

    
    pagina_actual = url_categoria #Asigna la URL inicial de la categoría a una variable que iremos actualizando
    libros = [] # Lista vacía en donde se va guardar un diccionario por cada libro

    while True:
        print(f"Scrapeando {pagina_actual}...")
        respuesta = requests.get(pagina_actual)
        soup = BeautifulSoup(respuesta.text, "html.parser")

        libros_en_pagina = soup.select(".product_pod")

        for libro in libros_en_pagina:
            titulo = libro.h3.a["title"] # Accede directamente a las etiquetas y entra directo a la etiqueta <a> y saca el atributo title desde el objeto libro 
            precio = libro.select_one(".price_color").text.strip() #Busca la primera etiqueta que tenga esa clase ".price_color" accede al texto dentro de la etiqueta y le saca espacios si tiene
            rating = libro.p["class"][1] # Accede a la segunda clase del rating
            libros.append({
                "categoria" : nombre_categoria,
                "titulo": titulo,
                "precio" : precio,
                "rating" : rating
            })

        next_button = soup.select_one("li.next a") # Busca si hay un enlace a la próxima página. Si existe, sigue
        # Construye la URL de la próxima página
        if next_button:
            next_url = next_button["href"]
            # Si pagina_actual = ".../index.html" y next_url = "page-2.html"➡ Reemplaza index.html con page-2.html
            if "index.html" in pagina_actual:
                pagina_actual = pagina_actual.replace("index.html", next_url )
            else:
                # Esta línea se ejecuta cuando hay más de una página en la categoría (ejemplo: page-2.html, page-3.html, etc.).
                pagina_actual = "/".join(pagina_actual.split("/")[:-1])+ "/" + next_url
        else: 
            break

        time.sleep(1) # Espera 1 segundo antes de pasar a la próxima página
    return libros # Devuelve la lista de diccionarios con todos los libros encontrados

# Visualización de los resultados

In [ ]:
#  lista vacía para juntar todos los libros del sitio
todos_los_libros = []
# Recorre todas las categorías que ya tenemos en el diccionario
for nombre,url in categoria.items():
    print(f"Scrapeando categoría: {nombre}...")

    libros_categoria = obtener_libros_por_categoria(nombre,url)

    todos_los_libros.extend(libros_categoria)  # agregamos todos los libros a la lista general
print(f"\nScraping completado. Total de libros encontrados: {len(todos_los_libros)}")